In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
from shared_utils import rt_utils, rt_dates

from rt_analysis import rt_parser, rt_filter_map_plot

import pandas as pd
import geopandas as gpd
from siuba import *
import datetime as dt

from tqdm.notebook import tqdm

In [ ]:
from calitp_data_analysis.tables import tbls

In [ ]:
mst_itp_id = 208

In [ ]:
key_dates = ["sep2022a", "oct2022", "nov2022", "dec2022",
"jan2023", "feb2023", "mar2023",
"apr2023", "may2023", "jun2023",
"jul2023",
"aug2023a"]

In [ ]:
# key_dates = key_dates[1:3]

In [ ]:
key_dates = [dt.date.fromisoformat(rt_dates.DATES[date]) for date in key_dates]
key_dates

In [ ]:
# [rt_utils.get_operators(date, [mst_itp_id]) for date in key_dates]

In [ ]:
def run_process(analysis_date):

    rt_day = rt_parser.OperatorDayAnalysis(mst_itp_id,
                                           analysis_date, pbar)


    rt_day.export_views_gcs()

    rt_day = rt_filter_map_plot.from_gcs(mst_itp_id, analysis_date, pbar)

    rt_day.set_filter(start_time='06:00', end_time='09:00')
    _m = rt_day.segment_speed_map()
    rt_day.set_filter(start_time='10:00', end_time='14:00')
    _m = rt_day.segment_speed_map()
    rt_day.set_filter(start_time='15:00', end_time='19:00')
    _m = rt_day.segment_speed_map()

In [ ]:
# for date in key_dates:
#     try:
#         run_process(date)
#     except Exception as e:
#         print(e)

In [ ]:
periods = {'am_peak': {'start_time': '06:00', 'end_time': '09:00'},
          'midday': {'start_time': '10:00', 'end_time': '14:00'},
           'pm_peak': {'start_time': '15:00', 'end_time': '19:00'}
          }

In [ ]:
pbar = tqdm()

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(mst_itp_id, key_dates[-1], pbar)

In [ ]:
rt_day.set_filter(route_names=['20'])

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
# _m

In [ ]:
rt_day.autocorridor(shape_id='0200095', stop_seq_range=[17, 24])

In [ ]:
static_corridor = rt_day.corridor.copy()

In [ ]:
# rt_day.segment_speed_map(corridor=True)

In [ ]:
all_dates = []

for date in key_dates:
    rt_day = rt_filter_map_plot.from_gcs(mst_itp_id, date, pbar)
    rt_day.transit_priority_target_mph = 60
    
    for period in periods.keys():
        rt_day.set_filter(**periods[period], route_names=['20'])
        _m = rt_day.segment_speed_map(no_render=True)
        rt_day.add_corridor(static_corridor)
        rt_day.corridor_metrics()
        rt_day.corridor["period"] = period
        rt_day.corridor["analysis_date"] = rt_day.analysis_date.isoformat()
        all_dates += [rt_day.corridor]
        
    rt_day.reset_filter()
    rt_day.set_filter(route_names=['20'])
    _m = rt_day.segment_speed_map(no_render=True)
    rt_day.add_corridor(static_corridor)
    rt_day.corridor_metrics()
    rt_day.corridor["period"] = "all_day"
    rt_day.corridor["analysis_date"] = rt_day.analysis_date.isoformat()
    all_dates += [rt_day.corridor]

In [ ]:
gdf = pd.concat(all_dates)

In [ ]:
gdf.to_parquet('./_mst_sep22_onwards.parquet')

In [ ]:
gdf >> head(5)

In [ ]:
gdf.columns

In [ ]:
(gdf >> select(_.route_id, _.route_short_name, _.organization,
              _.p20_corr_mph, _.p50_corr_mph, _.avg_corr_mph,
               _.n_trips,
              _.speed_delay_minutes, _.target_mph, _.period,
              _.analysis_date)).to_csv('./2022_23_marina_corridor_speeds.csv')

In [ ]:
rt_day.quick_map_corridor()